In [102]:
import rioxarray 
import xarray
import rasterio
import nasa_hls
import os
import urllib.request as urlreq
import pandas as pd
import fiona
import numpy as np
from glob import glob
from rasterio.enums import Resampling
from rasterio import Affine
from rasterio.crs import CRS
import matplotlib.pyplot as plt
from subprocess import Popen, PIPE

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline

In [96]:
##### START OPTIONS #####
yoi = [2020]
toi = ['15STT']
root_path = "/data/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
image_index_file = root_path + f"image_index{yoi[0]}.csv"
hdf_dir = root_path + "hdf/"
chip_geojson_dir = root_path + 'chip_geojson/'
chip_output_dir = root_path + 'chip_output/'
chip_cdl_dir = root_path + 'chip_cdl/'
#####  END OPTIONS  #####

make folders if needed

In [54]:
try:
    os.makedirs(hdf_dir)
except FileExistsError:
    # directory already exists
    print('pass')
    pass


pass


query and download hdf files

In [55]:
HLSdf = nasa_hls.get_available_datasets(
        years = yoi,
        products = ["S30"],
        tiles = toi,
        return_list = False)
        
#HLSdf.to_csv(spath, mode='w')

100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


In [56]:
for k in range(len(HLSdf)):
    url = HLSdf.at[k, "url"]
    local_name = url.split('/')[-1].replace("\n", "").replace('.hdf', '')
    HLSdf.at[k, "image_id"] = local_name
    try:
        urlreq.urlretrieve(url, filename = hdf_dir+local_name + '.hdf')
    except:
        print(local_name + " failed")
        continue
               
HLSdf.to_csv(spath, mode='w')

extract hdf metadata, filter to 3 scenes per tile

In [57]:
def get_metadata_from_hdf_mine(src, fields=["cloud_cover", "spatial_coverage"]):
    """Get metadata from a nasa-hls hdf file. See HLS user guide for valid fields.
    
    HLS User Guide - see Section 6.6: 
    
    https://hls.gsfc.nasa.gov/wp-content/uploads/2019/01/HLS.v1.4.UserGuide_draft_ver3.1.pdf
    """
    band="QA"
    cmd = f'gdalinfo HDF4_EOS:EOS_GRID:"{src}":Grid:{band}'
    print('cmd')
    print(cmd)
    p = Popen(cmd, stdout=PIPE, shell=True)
    output, err = p.communicate()
    output = str(output)[2:-1].replace("\\n", "\n")
    rc = p.returncode
    metadata = {}
    for line in output.split("\n"):
        for field in fields:
            if field in line:
                metadata[field] = line.split("=")[1].strip()
                try:
                    metadata[field] = float(metadata[field])
                except:
                    pass
    for field in fields:
        if field not in metadata.keys():
            warnings.warn(f"Could not find metadata for field '{field}'.")
    return metadata

In [ ]:
image_index = pd.DataFrame(columns = ['image_id', 'tile', 'date', 'month', 'cloud_coverage', 'spatial_coverage'])

candidate_hdf = sorted(glob(hdf_dir + '*.hdf'))

for img in candidate_hdf:
    print(img)
    local_name = img.split('/')[-1]
    try:
        print(hdf_dir+local_name)
        md = get_metadata_from_hdf_mine(hdf_dir+local_name)
    except:
        print(img + ' skipped')
        continue
    print(md)
    cloud_cover = int(md['cloud_cover'])
    spatial_coverage = int(md['spatial_coverage'])
    image_id = local_name.replace('.hdf', '')
    tname = local_name.split('.')[2]
    date = local_name.split('.')[3]
    image_date_string = image_id.split('.')[3]
    image_date = pd.to_datetime(image_date_string, format="%Y%j").date()
    image_month = image_date.month
    
    new_row = pd.DataFrame({'image_id':  [image_id],
               'tile': [tname],
               'date': [image_date],
               'month': [image_month],
               'cloud_coverage': [cloud_cover],
               'spatial_coverage': [spatial_coverage]})
    image_index = pd.concat([image_index, new_row], ignore_index = True)


image_index.to_csv(image_index_file)

In [105]:
image_index.sort_values(['cloud_coverage']).head(300)

,image_id,tile,date,month,cloud_coverage,spatial_coverage
7,HLS.S30.T15STT.2020020.v1.4,T15STT,2020-01-20,1,0,6
39,HLS.S30.T15STT.2020107.v1.4,T15STT,2020-04-16,4,1,100
1,HLS.S30.T15STT.2020005.v1.4,T15STT,2020-01-05,1,1,6
129,HLS.S30.T15STT.2020340.v1.4,T15STT,2020-12-05,12,1,5
16,HLS.S30.T15STT.2020045.v1.4,T15STT,2020-02-14,2,1,6
137,HLS.S30.T15STT.2020360.v1.4,T15STT,2020-12-25,12,1,6
106,HLS.S30.T15STT.2020280.v1.4,T15STT,2020-10-06,10,1,5
135,HLS.S30.T15STT.2020355.v1.4,T15STT,2020-12-20,12,1,5
108,HLS.S30.T15STT.2020285.v1.4,T15STT,2020-10-11,10,1,5
125,HLS.S30.T15STT.2020330.v1.4,T15STT,2020-11-25,11,1,5


Select 3 best images (need to loop this over tiles)

In [71]:
cloud_thres = 5
valid_months = [3,4,5,6,7,8,9]
cand_images = image_index[(image_index.spatial_coverage == 100) & (image_index.cloud_coverage <= cloud_thres) & (image_index.month.isin(valid_months))]
#cand_images.sort_values(['cloud_coverage']).head(50)


cand_image_count = len(cand_images)
first_image = cand_images.head(1)
last_image = cand_images.tail(1)
middle_image = cand_images.head(cand_image_count // 2).tail(1)

selected_images = pd.concat([first_image, middle_image, last_image], ignore_index = True)

,image_id,tile,date,month,cloud_coverage,spatial_coverage
39,HLS.S30.T15STT.2020107.v1.4,T15STT,2020-04-16,4,1,100
57,HLS.S30.T15STT.2020152.v1.4,T15STT,2020-05-31,5,2,100
61,HLS.S30.T15STT.2020162.v1.4,T15STT,2020-06-10,6,2,100
85,HLS.S30.T15STT.2020222.v1.4,T15STT,2020-08-09,8,3,100
91,HLS.S30.T15STT.2020237.v1.4,T15STT,2020-08-24,8,3,100
93,HLS.S30.T15STT.2020242.v1.4,T15STT,2020-08-29,8,3,100
103,HLS.S30.T15STT.2020272.v1.4,T15STT,2020-09-28,9,3,100


In [72]:
selected_images

,image_id,tile,date,month,cloud_coverage,spatial_coverage
0,HLS.S30.T15STT.2020107.v1.4,T15STT,2020-04-16,4,1,100
1,HLS.S30.T15STT.2020162.v1.4,T15STT,2020-06-10,6,2,100
2,HLS.S30.T15STT.2020272.v1.4,T15STT,2020-09-28,9,3,100


convert selected hdf to cog

In [73]:
def convert_hdf_to_cog(scene_id, product = "S30"):
    
    """
    This function receives the scene_id of an HLS scene (in a format similar to "HLS.S30.T14RNS.2020005.v1.4"
    and converts the scene from HDF format to COG. 
    
    Assumptions:
    - The corresponding HDF file for the scene is located at `/data/hdf/scene_id.hdf`
    - The output will be written to `/data/tif/scene_id/*.tif` and contains all the bands. 
    
    Inputs:
    - scene_id: The scene ID of the HLS scene
    - product: the HLS product ID. Default is S30, but it can be S30, L30, S30_ANGLES, L30_ANGLES
    
    """
    
    import os
    cmd = f"python3 /hls-hdf_to_cog/hls_hdf_to_cog/hls_hdf_to_cog.py --product {product} /data/hdf/{scene_id}.hdf --output-dir /data/tif/{scene_id}/"
    os.system(cmd)

In [74]:
for k in range(len(selected_images)):
    print(selected_images.at[k, 'image_id'])
    convert_hdf_to_cog(selected_images.at[k, 'image_id'])

HLS.S30.T15STT.2020107.v1.4
HLS.S30.T15STT.2020162.v1.4
HLS.S30.T15STT.2020272.v1.4


In [ ]:
convert_hdf_to_cog('HLS.S30.T15STT.2020032.v1.4')

reproject selected cog to cdl crs

In [ ]:
def reproject_hls_to_cdl(scene_folder,
                         bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"],
                         cdl_file = "/data/2022_30m_cdls_clipped.tif"):
    
    """
    This function receives the path to a folder that contains all GeoTIFF files (for various bands)
    of a HLS scene, and reprojects those to the target CDL CRS and grid. 
    
    Assumptions:
    - scene_folder has a file structure like: ".../<scene_id>/<scene_id>.<band_id>.tiff
    - scene_folder should not have a "/" at the end
    
    Inputs:
    - scene_folder: is the path to the folder that contains HLS GeoTIFF files for all bands of HLS
    - bands: list of bands of HLS that should be reprojected (default is all bands)
    - cdl_file: contains the path to the clipped CDL GeoTIFF file
    
    """
    
    for band in bands:
        xds = xarray.open_rasterio(f"{scene_folder}/{scene_folder.split('/')[-1]}.{band}.tif")
        cdl = xarray.open_rasterio(cdl_file)
        xds_new = xds.rio.reproject_match(cdl, resampling = Resampling.bilinear)
        xds_new.rio.to_raster(raster_path = f"{scene_folder}/{scene_folder.split('/')[-1]}.{band}.5070.tif")

In [ ]:
for k in range(len(selected_images):
    image_id = selected_images.at[k, 'image_id']
    print(image_id)
    reproject_hls_to_cdl("/data/tif/" + image_id,
                         bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"])

HLS.S30.T15STT.2020107.v1.4
HLS.S30.T15STT.2020162.v1.4
HLS.S30.T15STT.2020272.v1.4


chipping

In [99]:
# this cell is a sample clipping of an HLS scene using a sample file `chip.geojson`

chips_list = sorted(glob(chip_geojson_dir + '*.geojson'))
print(chips_list)


['/data/chip_geojson/test_chip.geojson', '/data/chip_geojson/test_chip_2.geojson']


In [100]:


for k in range(len(chips_list)):
    chip = chips_list[k]
    ## get chip id
    chip_id = k
    
    ## get chip tile
    chip_tile = '15STT'
    
    ## get reprojected image paths
    selected_image_folders = sorted(glob(f'/data/tif/*{chip_tile}*'))
    print(selected_image_folders)
    
    assert len(selected_image_folders) == 3
    
    with fiona.open(chip, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    
    bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"]
    first_date_images = sorted(glob(selected_image_folders[1] + '/*.5070.tif'))
    second_date_images = sorted(glob(selected_image_folders[1] + '/*.5070.tif'))
    third_date_images = sorted(glob(selected_image_folders[2] + '/*.5070.tif'))
    all_date_images = first_date_images + second_date_images + third_date_images
    
    assert len(all_date_images) == 3 * len(bands)

    ## do we want to scale/clip reflectances?
    out_bands = []
    
    for img in all_date_images:
        with rasterio.open(img) as src:
            out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
            out_meta = src.meta
            out_bands.append(out_image[0])
    
    out_bands = np.array(out_bands)
    print(out_bands.shape)
    print(out_image.shape)
    ## band order?   
    out_meta.update({"driver": "GTiff",
                     "height": out_bands.shape[1],
                     "width": out_bands.shape[2],
                     "count": out_bands.shape[0],
                     "transform": out_transform})
    print(out_meta)
    with rasterio.open(chip_output_dir + "chip_hls_id_" + str(chip_id) + ".tif", "w", **out_meta) as dest:
        dest.write(out_bands)

 
  ## clip cdl
  
    with rasterio.open("/data/2022_30m_cdls_clipped.tif") as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta
     #   colormap = src.colormap(1)

    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(chip_cdl_dir + "chip_cdl_id_" + str(chip_id) + ".tif", "w", **out_meta) as dest:
        dest.write(out_image)
       # dest.write_colormap(1, colormap)



['/data/tif/HLS.S30.T15STT.2020107.v1.4', '/data/tif/HLS.S30.T15STT.2020162.v1.4', '/data/tif/HLS.S30.T15STT.2020272.v1.4']
(39, 8, 9)
(1, 8, 9)
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -1000.0, 'width': 9, 'height': 8, 'count': 39, 'crs': CRS.from_wkt('PROJCS["NAD83 / Conus Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(30.0, 0.0, -12225.0,
       0.0, -30.0, 1218555.0)}
['/data/tif/HLS.S30.T15STT.2020107.v1.4', '/data/t

[2]